In [1]:
import numpy as np
import pandas as pd

In [2]:
data = pd.read_csv("train.csv")
data_test = pd.read_csv("test.csv")

# Encoding

In [3]:
data['internet_access'] = data['internet_access'].map({'no': 0, 'yes': 1})
data['facility_rating'] = data['facility_rating'].map({'low': 0, 'medium': 1, 'high': 2})
data['exam_difficulty'] = data['exam_difficulty'].map({'easy': 0, 'moderate': 1, 'hard': 2})
data['sleep_quality'] = data['sleep_quality'].map({'poor': 0, 'average': 1, 'good': 2})
data_test['internet_access'] = data_test['internet_access'].map({'no': 0, 'yes': 1})
data_test['facility_rating'] = data_test['facility_rating'].map({'low': 0, 'medium': 1, 'high': 2})
data_test['exam_difficulty'] = data_test['exam_difficulty'].map({'easy': 0, 'moderate': 1, 'hard': 2})
data_test['sleep_quality'] = data_test['sleep_quality'].map({'poor': 0, 'average': 1, 'good': 2})
# Create dummy variables for nominal columns
data = pd.get_dummies(data, columns=['gender', 'course', 'study_method'], drop_first=True)
data_test = pd.get_dummies(data_test, columns=['gender', 'course', 'study_method'], drop_first=True)

# Regularization

In [ ]:
data.corr()["exam_score"]

id                            0.000372
age                           0.010472
study_hours                   0.762267
class_attendance              0.360954
internet_access               0.000447
sleep_hours                   0.167410
sleep_quality                 0.236895
facility_rating               0.188159
exam_difficulty               0.008773
exam_score                    1.000000
gender_male                  -0.012403
gender_other                  0.010302
course_b.sc                  -0.005178
course_b.tech                 0.000353
course_ba                    -0.010823
course_bba                    0.014164
course_bca                    0.001703
course_diploma               -0.001587
study_method_group study     -0.051422
study_method_mixed            0.067598
study_method_online videos   -0.071656
study_method_self-study      -0.130269
Name: exam_score, dtype: float64

In [ ]:
data = data.drop(["internet_access","id","gender_male","gender_other"],axis=1)
data_test = data_test.drop(["internet_access","id","gender_male","gender_other"],axis=1)


In [ ]:
def data_meanization(data,data_test):
    new_data_test =data_test/data_test.mean()
    new_data =data/data.mean()
    return new_data,new_data_test

# Correlation <0.1

In [ ]:
def corr_garb(new_data,new_data_test,X,X_t):
    correlation = new_data.corr()["exam_score"]<0.1
    correlation_test = correlation.drop("exam_score")
    print(correlation)
    for i in new_data_test:
        if(correlation_test[i]==True):
            X= X.drop(i,axis=1)
            X_t = X_t.drop(i,axis=1)
    return X,X_t

# Multiply with correlation

In [ ]:
def corr_prod(new_data,new_data_test,X,X_t):
    correlation = new_data.corr()["exam_score"]
    correlation_test = correlation.drop("exam_score")
    for i in new_data_test:
        X = X*correlation
        X_t = X_t*correlation_test
    return X,X_t

In [ ]:
X,X_t = data_meanization(data,data_test)
#X,X_t = corr_prod(new_data,new_data_test,X,X_t)
#X,X_t = corr_garb(new_data,new_data_test,X,X_t)

In [ ]:
import tensorflow as tf
y=X["exam_score"]
X = X.drop(["exam_score"],axis=1)
# Convert everything to float32
X = X.astype('float32')
y = y.astype('float32')
X_t = X_t.astype("float32")

2026-02-24 14:03:20.843071: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1771941801.067151      24 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1771941801.133358      24 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1771941801.655670      24 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1771941801.655727      24 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1771941801.655730      24 computation_placer.cc:177] computation placer alr

# Train Test Split

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
X, y = shuffle(X, y, random_state=42)
# Split: 80% for training, 20% for testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Training shape: {X_train.shape}")
print(f"Testing shape: {X_test.shape}")


Training shape: (504000, 17)
Testing shape: (126000, 17)


# Linear Regression

In [ ]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(X_train,y_train)
y_pred = model.predict(X_test)
y_pred


array([1.2846162, 0.7948787, 0.8223005, ..., 1.1569989, 1.229423 ,
       1.1965537], dtype=float32)

# XGBOOST

In [ ]:
import xgboost as xgb

model_xg = xgb.XGBRegressor(objective='reg:squarederror',
                         n_estimators=1000, random_state=42)
model_xg.fit(X_train, y_train)
y_pred_xg = model_xg.predict(X_test)
y_pred_xg

array([1.2165458 , 0.79068744, 0.83533585, ..., 1.130766  , 1.2503719 ,
       1.1641605 ], dtype=float32)

# GRADIENT BOOSTING

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
model_gradient = GradientBoostingRegressor(loss="squared_error")
model_gradient.fit(X_train,y_train)
y_pred_grad = model_gradient.predict(X_test)
y_pred_grad

array([1.2445089 , 0.78508924, 0.81203654, ..., 1.16885515, 1.22019298,
       1.1785081 ])

# LASSO

In [ ]:
from sklearn.linear_model import Lasso
model_lasso = Lasso()
model_lasso.fit(X_train,y_train)
y_pred_lasso = model_lasso.predict(X_test)
y_pred_lasso

array([0.9998359, 0.9998359, 0.9998359, ..., 0.9998359, 0.9998359,
       0.9998359], dtype=float32)

In [ ]:
from sklearn.linear_model import RidgeCV

ridge = RidgeCV(alphas=[1])
ridge.fit(X_train, y_train)
y_pred_ridge = ridge.predict(X_test)


# TENSORFLOW

In [ ]:
# Define batch size (adjust based on your RAM, 32 or 64 is standard)
BATCH_SIZE = 64

# Create Training Dataset
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train.values))
train_dataset = train_dataset.shuffle(buffer_size=10000).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

# Create Testing Dataset (No need to shuffle)
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test.values))
test_dataset = test_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)


I0000 00:00:1771941892.935787      24 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13757 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1771941892.941409      24 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13757 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


In [ ]:
test_dataset1 = tf.data.Dataset.from_tensor_slices((X_t))
test_dataset1 = test_dataset1.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

In [ ]:
# Take just one batch
X_sample, y_sample = next(iter(train_dataset))

# Create a tiny model (overkill for 64 rows, but good for testing)
debug_model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)
])
debug_model.compile(optimizer='adam', loss='mse')

# Train for many epochs on just this tiny data
print("Attempting to overfit one batch...")
debug_model.fit(X_sample, y_sample, epochs=100, verbose=0)
final_loss = debug_model.evaluate(X_sample, y_sample, verbose=0)

print(f"Final Loss on 1 batch: {final_loss}")

Attempting to overfit one batch...


I0000 00:00:1771941893.993041      71 service.cc:152] XLA service 0x7f411c007ac0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1771941893.993097      71 service.cc:160]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1771941893.993102      71 service.cc:160]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1771941894.331701      71 cuda_dnn.cc:529] Loaded cuDNN version 91002
I0000 00:00:1771941895.234050      71 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


Final Loss on 1 batch: 0.009510558098554611


In [ ]:
model_early = tf.keras.Sequential([
    tf.keras.layers.Dense(1024, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)
])
opt = tf.keras.optimizers.Adam(learning_rate=0.01)
initial_weights = model_early.get_weights()
model_early.set_weights(initial_weights)
model_early.compile(optimizer=opt, loss='mse', metrics=['mae']) # Örnek compile
history1 = model_early.fit(
    train_dataset,
    epochs=100,
    validation_data=test_dataset,
    callbacks=[tf.keras.callbacks.EarlyStopping()] # Sizin yazdığınız callback'i buraya ekleyebilirsiniz
)


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/100
7875/7875 ━━━━━━━━━━━━━━━━━━━━ 22s 3ms/step - loss: 0.3060 - mae: 0.1468 - val_loss: 0.0210 - val_mae: 0.1154
Epoch 2/100
7875/7875 ━━━━━━━━━━━━━━━━━━━━ 20s 2ms/step - loss: 0.0211 - mae: 0.1161 - val_loss: 0.0204 - val_mae: 0.1141
Epoch 3/100
7875/7875 ━━━━━━━━━━━━━━━━━━━━ 20s 3ms/step - loss: 0.0210 - mae: 0.1158 - val_loss: 0.0207 - val_mae: 0.1147


In [ ]:
new_test = model_early.predict(test_dataset)

1969/1969 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step


# RESULTS

In [ ]:
from sklearn.metrics import mean_squared_error
linear_result= np.sqrt(mean_squared_error(y_pred,y_test))
tf_result = np.sqrt(mean_squared_error(new_test,y_test))
xg_result = np.sqrt(mean_squared_error(y_pred_xg,y_test))
grad_result = np.sqrt(mean_squared_error(y_pred_grad,y_test))
lasso_result = np.sqrt(mean_squared_error(y_pred_lasso,y_test))
ridge_result = np.sqrt(mean_squared_error(y_pred_ridge,y_test))

print(f" Linear Result: {linear_result}\n Tensorflow Result: {tf_result}\n XGBRegressor: {xg_result}\n GradientBoostingRegressor:{grad_result}\n Lasso: {lasso_result}\n Ridge: {ridge_result}")

 Linear Result: 0.14231032866889273
 Tensorflow Result: 0.1438974065268915
 XGBRegressor: 0.1422562100428464
 GradientBoostingRegressor:0.14230799572798647
 Lasso: 0.3024680042488213
 Ridge: 0.14230892838644008


# START OF SUBMMISSION

In [ ]:
new= model.predict(X_t)

In [ ]:
new = new*data["exam_score"].mean()
new

array([71.82332129, 69.6231237 , 87.57801887, ..., 90.25802547,
       55.45853857, 68.42109753])

In [ ]:
a = [i+630000 for i in range(270000)]
a = np.asarray(a)
a

array([630000, 630001, 630002, ..., 899997, 899998, 899999])

In [ ]:
a.reshape(-1,1)
a
new.reshape(-1,1)

array([[71.82332129],
       [69.6231237 ],
       [87.57801887],
       ...,
       [90.25802547],
       [55.45853857],
       [68.42109753]])

In [ ]:
df = pd.DataFrame(
    data = new)


In [ ]:
df["id"] = a

In [ ]:
df["exam_score"] = new

In [ ]:
df

,0,id,exam_score
0,71.823321,630000,71.823321
1,69.623124,630001,69.623124
2,87.578019,630002,87.578019
3,54.907964,630003,54.907964
4,47.135031,630004,47.135031
...,...,...,...
269995,60.522188,899995,60.522188
269996,38.895307,899996,38.895307
269997,90.258025,899997,90.258025
269998,55.458539,899998,55.458539


In [ ]:
df = df.drop(0,axis=1)
df

,id,exam_score
0,630000,71.823321
1,630001,69.623124
2,630002,87.578019
3,630003,54.907964
4,630004,47.135031
...,...,...
269995,899995,60.522188
269996,899996,38.895307
269997,899997,90.258025
269998,899998,55.458539


In [ ]:
df.to_csv("sub.csv",index=False)

# EXTRA FUZZY LOGIC TRY WITH AI HELP

In [ ]:
import tensorflow as tf
import numpy as np

class FuzzyEarlyStopping(tf.keras.callbacks.Callback):
    def __init__(self, patience=0):
        super(FuzzyEarlyStopping, self).__init__()
        self.patience = patience  # Acil durumlar için klasik sayaç (yedek)
        self.wait = 0
        self.best_weights = None
        self.best_loss = np.inf

    def _membership_triangle(self, x, a, b, c):
        """Basit Üçgen Üyelik Fonksiyonu"""
        return max(min((x - a) / (b - a), (c - x) / (c - b)), 0) if b != a and c != b else 0

    def _membership_grade(self, x, start, end):
        """Yarı-Rampa Üyelik (0'dan 1'e artış)"""
        if x <= start: return 0.0
        if x >= end: return 1.0
        return (x - start) / (end - start)

    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        val_loss = logs.get('val_loss')
        train_loss = logs.get('loss')

        if val_loss is None or train_loss is None:
            return

        # 1. GİRİŞLER (Crisp Inputs)
        # ---------------------------------------------------------
        # Delta: Doğrulama hatası ne kadar değişti? (Pozitifse kötü)
        delta = val_loss - self.best_loss

        # Gap: Overfitting miktarı (Val ile Train arasındaki makas)
        gap = val_loss - train_loss

        # En iyi kaybı güncelle (Klasik mantık takibi için)
        if val_loss < self.best_loss:
            self.best_loss = val_loss
            self.best_weights = self.model.get_weights()
            self.wait = 0
        else:
            self.wait += 1

        # 2. BULANIKLAŞTIRMA (Fuzzification)
        # ---------------------------------------------------------
        # Gap (Makas) Üyelikleri:
        # 0.02 altı güvenli, 0.10 üstü tehlikeli (Veri setine göre ayarlanabilir)
        mu_gap_low = 1.0 - self._membership_grade(gap, 0.0, 0.05)
        mu_gap_high = self._membership_grade(gap, 0.05, 0.15)

        # Trend (Gidişat) Üyelikleri:
        # Delta negatifse iyileşiyor, pozitifse kötüleşiyor
        mu_trend_improving = 1.0 if delta < 0 else 0.0
        mu_trend_worsening = self._membership_grade(delta, 0.0, 0.02) # Biraz artışa tolerans

        # 3. KURAL TABANI VE ÇIKARIM (Inference)
        # ---------------------------------------------------------
        # Risk Skoru Hesabı (0: Güvenli, 1: Durdur)

        risk_score = 0.0

        # KURAL 1: Gidişat kötüleşiyor VE Makas çok açıksa -> YÜKSEK RİSK
        rule1_strength = min(mu_trend_worsening, mu_gap_high)
        if rule1_strength > 0:
            risk_score = max(risk_score, 1.0) # Kesin Durdur kararı

        # KURAL 2: Gidişat kötüleşiyor ama Makas az ise -> ORTA RİSK
        rule2_strength = min(mu_trend_worsening, mu_gap_low)
        if rule2_strength > 0:
            risk_score = max(risk_score, 0.6) # Yavaşla kararı

        # KURAL 3: Sadece makas açılıyorsa (Trend iyi olsa bile) -> DÜŞÜK RİSK
        if mu_gap_high > 0.8:
            risk_score = max(risk_score, 0.4)

        # 4. DURULAŞTIRMA VE EYLEM (Defuzzification & Action)
        # ---------------------------------------------------------
        current_lr = float(tf.keras.backend.get_value(self.model.optimizer.learning_rate))

        print(f"\n[Fuzzy Logic] Epoch: {epoch+1} | Gap: {gap:.4f} | Risk: {risk_score:.2f}")

        if risk_score >= 0.9:
            print(">>> KARAR: ACİL DURDURMA (Overfitting Tespit Edildi)")
            self.model.stop_training = True
            self.model.set_weights(self.best_weights) # En iyiye dön

        elif risk_score >= 0.5:
            new_lr = current_lr * 0.5
            print(f">>> KARAR: YAVAŞLA (LR: {current_lr:.1e} -> {new_lr:.1e})")

            # ESKİSİ (Hata veren):
            # tf.keras.backend.set_value(self.model.optimizer.learning_rate, new_lr)

            # YENİSİ (Doğrusu):
            try:
                # Çoğu TensorFlow sürümü için
                self.model.optimizer.learning_rate.assign(new_lr)
            except AttributeError:
                # Keras 3 veya farklı backend durumları için yedek
                self.model.optimizer.learning_rate = new_lr

        else:
            print(">>> KARAR: DEVAM (Her şey yolunda)")

        # Emniyet Sübabı: Eğer bulanık mantık kaçırırsa klasik patience devreye girsin
        if self.wait >= 10:
            print(">>> Emniyet Kilidi: Klasik Patience limiti aşıldı.")
            self.model.stop_training = True
            self.model.set_weights(self.best_weights)

In [ ]:
model_fuzzy = tf.keras.Sequential([
    tf.keras.layers.Dense(1024, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)
])
opt = tf.keras.optimizers.Adam(learning_rate=0.01)
model_fuzzy.compile(optimizer=opt, loss='mse', metrics=['mae']) # Örnek compile
model_fuzzy.set_weights(initial_weights)
fuzzy_callback = FuzzyEarlyStopping(patience=10)
history = model_fuzzy.fit(
    train_dataset,
    epochs=10,
    validation_data=test_dataset,
    callbacks=[fuzzy_callback] # Sizin yazdığınız callback'i buraya ekleyebilirsiniz
)

Epoch 1/10


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


7862/7875 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.3228 - mae: 0.1475
[Fuzzy Logic] Epoch: 1 | Gap: -0.0364 | Risk: 0.00
>>> KARAR: DEVAM (Her şey yolunda)
7875/7875 ━━━━━━━━━━━━━━━━━━━━ 22s 3ms/step - loss: 0.3223 - mae: 0.1475 - val_loss: 0.0207 - val_mae: 0.1150
Epoch 2/10
7855/7875 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0211 - mae: 0.1161
[Fuzzy Logic] Epoch: 2 | Gap: -0.0006 | Risk: 0.00
>>> KARAR: DEVAM (Her şey yolunda)
7875/7875 ━━━━━━━━━━━━━━━━━━━━ 20s 2ms/step - loss: 0.0211 - mae: 0.1161 - val_loss: 0.0205 - val_mae: 0.1142
Epoch 3/10
7864/7875 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0209 - mae: 0.1156
[Fuzzy Logic] Epoch: 3 | Gap: -0.0003 | Risk: 0.60
>>> KARAR: YAVAŞLA (LR: 1.0e-02 -> 5.0e-03)
7875/7875 ━━━━━━━━━━━━━━━━━━━━ 19s 2ms/step - loss: 0.0209 - mae: 0.1156 - val_loss: 0.0206 - val_mae: 0.1144
Epoch 4/10
7866/7875 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0205 - mae: 0.1145
[Fuzzy Logic] Epoch: 4 | Gap: -0.0003 | Risk: 0.00
>>> KARAR: DEVAM (Her şey yolun

In [ ]:
results = model_fuzzy.evaluate(test_dataset, verbose=0)
print('Test loss, Test accuracy:', results)

Test loss, Test accuracy: [0.020247263833880424, 0.11349408328533173]
